<a href="https://colab.research.google.com/github/erikaduda/dsto-gan/blob/main/DSTO_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**DSTO GAN**


O algoritmo DSTO-GAN consiste em uma implementação adaptada do DeepSMOTE para dados tabulares, que integra um modelo DeepSMOTE com uma arquitetura de Rede Generativa Adversarial (GAN) para o balanceamento de conjuntos de dados. Especificamente, o DSTO-GAN emprega um Discriminador com o objetivo de aprimorar a qualidade das amostras sintéticas geradas durante o processo. O treinamento da GAN é conduzido de forma iterativa, alternando-se entre a otimização do Discriminador e do Gerador em cada época. As amostras sintéticas são produzidas pelo Decoder da GAN, que é responsável por mapear os dados do espaço latente para o espaço de características, garantindo a geração de exemplos sintéticos que preservam a distribuição original dos dados. Essa abordagem visa melhorar a representatividade das classes minoritárias, contribuindo para a eficácia de modelos de aprendizado de máquina em cenários desbalanceados.

### Melhorias Implementadas no Algoritmo DeepSMOTE com GAN
Neste trabalho, propõem-se uma série de melhorias ao algoritmo DeepSMOTE com GAN, visando aprimorar a qualidade das amostras sintéticas geradas, o desempenho do modelo e a robustez do processo de balanceamento de dados. As principais alterações e implementações são descritas a seguir:


#### 1. Arquitetura do GAN
Foi adicionada uma rede Discriminador à arquitetura GAN, com o objetivo de melhorar a qualidade das amostras sintéticas geradas pelo Decoder. O Discriminador atua como um crítico, avaliando a veracidade das amostras sintéticas em relação aos dados reais. Essa abordagem adversarial permite que o Gerador aprenda a produzir amostras mais realistas, aproximando-se da distribuição original dos dados. A interação entre o Gerador e o Discriminador é fundamental para garantir que as amostras sintéticas sejam representativas e úteis para o balanceamento de classes.

#### 2. Função G_SM1
A função G_SM1, responsável pela geração de amostras sintéticas, foi reformulada para incorporar o uso do GAN. Agora, em vez de depender exclusivamente de técnicas tradicionais de oversampling, a função utiliza o Gerador da GAN para criar amostras sintéticas a partir do espaço latente. Essa mudança permite a geração de dados mais diversificados e adaptados à distribuição dos dados reais, melhorando a eficácia do balanceamento.

#### 3. Treinamento do GAN
O treinamento da GAN foi implementado de forma iterativa, alternando entre a otimização do Gerador e do Discriminador. Em cada época, o Discriminador é treinado para distinguir entre amostras reais e sintéticas, enquanto o Gerador é ajustado para enganar o Discriminador, produzindo amostras cada vez mais realistas. Esse processo adversarial é repetido até que o Gerador consiga gerar amostras sintéticas de alta qualidade, que sejam indistinguíveis das reais pelo Discriminador.

#### 4. Pipeline de Validação Cruzada
Foi implementado um pipeline de validação cruzada estratificada para garantir uma avaliação robusta e imparcial do modelo. A validação cruzada estratificada preserva a proporção das classes em cada fold, evitando vieses na avaliação. Durante o processo, métricas como F1-Score, Recall e Precision são coletadas e analisadas, permitindo uma avaliação detalhada do desempenho do modelo em diferentes cenários.

#### 5. Salvamento e Carregamento de Modelos
O processo de salvamento e carregamento dos modelos GAN e dos classificadores foi ajustado para garantir a reprodutibilidade dos experimentos e a facilidade de uso em diferentes contextos. Agora, tanto o modelo GAN (incluindo o Gerador e o Discriminador) quanto os classificadores treinados podem ser salvos em arquivos e carregados posteriormente para inferência ou continuidade do treinamento. Essa funcionalidade é essencial para aplicações práticas, onde modelos pré-treinados podem ser reutilizados sem a necessidade de retreinamento.

#### 6. Otimizações Adicionais para Maximização de Métricas
Para otimizar ainda mais o algoritmo DeepSMOTE com GAN e maximizar as métricas de avaliação, como F1-Score, Recall e Precision, propõem-se as seguintes melhorias e ajustes:

a) **Otimização de Hiperparâmetros**: Realizar ajustes finos nos hiperparâmetros do modelo, como a dimensão do espaço latente ((n_z)), o número de épocas de treinamento ((epochs)) e as taxas de aprendizado ((lr)), visando aprimorar o desempenho geral do algoritmo. A escolha adequada desses parâmetros é crucial para garantir a convergência e a eficácia do modelo.

b) **Refinamento da Arquitetura do Modelo**: Explorar diferentes configurações arquiteturais para o Encoder e o Decoder, incluindo a adição de camadas adicionais ou a utilização de funções de ativação alternativas (e.g., ReLU, LeakyReLU, tanh). Essas modificações podem melhorar a capacidade de representação do modelo e a qualidade das amostras sintéticas geradas.

c) **Técnicas de Aumento de Dados**: Incorporar métodos de aumento de dados para enriquecer a diversidade das amostras sintéticas. Isso pode incluir a aplicação de transformações ou perturbações controladas nos dados, aumentando a robustez do modelo e sua capacidade de generalização.

d) **Avaliação de Desempenho**: Implementar uma avaliação abrangente utilizando diferentes classificadores (e.g., Random Forest, SVM, Redes Neurais) para analisar o impacto das amostras sintéticas geradas. Realizar uma análise detalhada das métricas de desempenho, como F1-Score, Recall e Precision, para validar a eficácia do balanceamento e a qualidade dos dados sintéticos.

### Conclusão
As melhorias implementadas no algoritmo DeepSMOTE com GAN visam aumentar a qualidade das amostras sintéticas geradas, melhorar o desempenho do modelo em tarefas de classificação e garantir a robustez do processo de balanceamento de dados. A adição do Discriminador, a reformulação da função G_SM1, o treinamento adversarial da GAN, a inclusão do XGBoost, a implementação de validação cruzada estratificada e o ajuste no salvamento e carregamento de modelos contribuem para um framework mais eficiente e aplicável em cenários reais. Além disso, as otimizações adicionais, como ajuste de hiperparâmetros, refinamento da arquitetura, técnicas de aumento de dados e avaliação de desempenho, permitem que o algoritmo lide de forma mais eficaz com conjuntos de dados desbalanceados, resultando em modelos de aprendizado de máquina mais precisos e generalizáveis. Essas alterações consolidam o DeepSMOTE com GAN como uma ferramenta robusta e versátil para o tratamento de dados desbalanceados em diversas aplicações.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install bayesian-optimization

In [ ]:
pip install scikit-optimize

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 3.0 MB/s eta 0:00:00


In [ ]:
!pip uninstall torch torchvision torchaudio -y
!pip install torch torchvision torchaudio

Found existing installation: torch 2.5.1+cu121
Uninstalling torch-2.5.1+cu121:
  Successfully uninstalled torch-2.5.1+cu121
Found existing installation: torchvision 0.20.1+cu121
Uninstalling torchvision-0.20.1+cu121:
  Successfully uninstalled torchvision-0.20.1+cu121
Found existing installation: torchaudio 2.5.1+cu121
Uninstalling torchaudio-2.5.1+cu121:
  Successfully uninstalled torchaudio-2.5.1+cu121
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
import torch.optim as optim
import os
import torch.optim as optim
import joblib
from torch.utils.data import TensorDataset
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from skopt import BayesSearchCV


# Definir parâmetros para os classificadores
param_spaces = {
    'Decision Tree': {'max_depth': (3, 15)},
    'Random Forest': {'n_estimators': (10, 100)},
    'Neural Network': {'alpha': (1e-4, 1e-2, 'log-uniform')},
    'KNN': {'n_neighbors': (3, 10)},
    'XGBoost': {'learning_rate': (0.01, 0.3)}
}


# Definir argumentos para o modelo
args = {
    'dim_h': 64,
    'n_z': 10,
    'lr': 0.0002,
    'epochs': 100,
    'batch_size': 64,
    'save': True,
    'train': True
}

class Encoder(nn.Module):
    def __init__(self, args, num_input_features):
        super(Encoder, self).__init__()
        self.dim_h = args['dim_h']
        self.n_z = args['n_z']
        self.fc1 = nn.Linear(num_input_features, self.dim_h)
        self.fc2 = nn.Linear(self.dim_h, self.dim_h)
        self.fc_mean = nn.Linear(self.dim_h, self.n_z)
        self.fc_logvar = nn.Linear(self.dim_h, self.n_z)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        mean = self.fc_mean(x)
        logvar = self.fc_logvar(x)
        return mean, logvar

class Decoder(nn.Module):
    def __init__(self, args, num_input_features):
        super(Decoder, self).__init__()
        self.dim_h = args['dim_h']
        self.n_z = args['n_z']
        self.fc1 = nn.Linear(self.n_z, self.dim_h)
        self.fc2 = nn.Linear(self.dim_h, self.dim_h)
        self.fc_output = nn.Linear(self.dim_h, num_input_features)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc_output(x)
        return x

class Discriminator(nn.Module):
    def __init__(self, num_input_features):
        super(Discriminator, self).__init__()
        self.fc1 = nn.Linear(num_input_features, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 1)
        self.relu = nn.ReLU(inplace=True)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.sigmoid(x)
        return x

def G_SM1(X, y, n_to_sample, cl, encoder, decoder):
    # Genera amostras sintéticas usando o GAN
    X_tensor = torch.tensor(X, dtype=torch.float32)
    y_tensor = torch.tensor(y, dtype=torch.long)
    dataloader = torch.utils.data.DataLoader(TensorDataset(X_tensor, y_tensor), batch_size=args['batch_size'], shuffle=True)

    synthetic_data = []
    for _ in range(n_to_sample):
        z = torch.randn(1, args['n_z'])
        synthetic_sample = decoder(z).detach().numpy()
        synthetic_data.append(synthetic_sample)

    synthetic_data = np.vstack(synthetic_data)
    synthetic_labels = np.array([cl] * n_to_sample)
    return synthetic_data, synthetic_labels

def calculate_n_to_sample(y):
    class_counts = np.bincount(y)
    major_class_count = np.max(class_counts)
    n_classes = len(class_counts)
    n_to_sample_dict = {cl: major_class_count - class_counts[cl] for cl in range(n_classes)}
    return n_to_sample_dict, major_class_count

# Diretório de entrada e saída
INPUT_DIR = "/content/drive/MyDrive/PHD_new/dataset_tratado/fast/"
OUTPUT_DIR_TRAIN = "/content/drive/MyDrive/PHD_new/resultados/dsto_gan/treino/"
OUTPUT_DIR_TEST = "/content/drive/MyDrive/PHD_new/resultados/dsto_gan/teste/"
MODELO_DIR = "/content/drive/MyDrive/PHD_new/resultados/dsto_gan/modelos/"
MODELO_DIR_DST = "/content/drive/MyDrive/PHD_new/resultados/dsto_gan/modelos_dsto/"

# Verificar se o diretório de saída existe, se não, criar
os.makedirs(OUTPUT_DIR_TRAIN, exist_ok=True)
os.makedirs(OUTPUT_DIR_TEST, exist_ok=True)
os.makedirs(MODELO_DIR, exist_ok=True)
os.makedirs(MODELO_DIR_DST, exist_ok=True)

# Listar arquivos .csv no diretório de entrada
csv_files = [f for f in os.listdir(INPUT_DIR) if f.endswith('.csv')]

# Iterar sobre os arquivos .csv
for csv_file in csv_files:
    input_path = os.path.join(INPUT_DIR, csv_file)
    data = pd.read_csv(input_path)
    X = data.drop('class', axis=1).values
    y = data['class'].values
    print(f"Processing: {csv_file}")
    print(f"CLASS DISTRIBUTION:\n{data['class'].value_counts(normalize=True) * 100}\n")

    # Dividir os dados em conjunto de treinamento e validação
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

    # Inicializar o DataFrame para armazenar as métricas de avaliação
    reports_df_train = pd.DataFrame(columns=['Classifier', 'Class', 'Fold', 'Precision', 'Recall', 'F1-Score'])
    reports_df_val = pd.DataFrame(columns=['Classifier', 'Class', 'Precision', 'Recall', 'F1-Score'])


    # Define classifiers
    classifiers = {
        'Decision Tree': DecisionTreeClassifier(),
        'Random Forest': RandomForestClassifier(),
        'Neural Network': MLPClassifier(),
        'KNN': KNeighborsClassifier(),
        'XGBoost': XGBClassifier()  # Adicionando XGBClassifier
    }

    # Inicializar listas para armazenar as métricas de validação de cada classificador
    precision_val_list = {classifier_name: [] for classifier_name in classifiers}
    recall_val_list = {classifier_name: [] for classifier_name in classifiers}
    f1_val_list = {classifier_name: [] for classifier_name in classifiers}

    # Realizar validação cruzada estratificada de 10 folds
    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

    for fold, (train_index, test_index) in enumerate(skf.split(X_train, y_train), 1):
        X_fold_train, X_fold_test = X_train[train_index], X_train[test_index]
        y_fold_train, y_fold_test = y_train[train_index], y_train[test_index]

        # Treinar o GAN
        if args['train']:
            encoder = Encoder(args, X_fold_train.shape[1])
            decoder = Decoder(args, X_fold_train.shape[1])
            discriminator = Discriminator(X_fold_train.shape[1])
            optimizer_g = optim.Adam(list(encoder.parameters()) + list(decoder.parameters()), lr=args['lr'])
            optimizer_d = optim.Adam(discriminator.parameters(), lr=args['lr'])
            criterion_g = nn.MSELoss()
            criterion_d = nn.BCELoss()

            dataloader = torch.utils.data.DataLoader(torch.tensor(X_fold_train, dtype=torch.float32), batch_size=args['batch_size'], shuffle=True)

            for epoch in range(args['epochs']):
                for batch in dataloader:
                    # Treinar o discriminador
                    optimizer_d.zero_grad()
                    real_labels = torch.ones(batch.size(0), 1)
                    fake_labels = torch.zeros(batch.size(0), 1)

                    outputs = discriminator(batch)
                    d_loss_real = criterion_d(outputs, real_labels)

                    z = torch.randn(batch.size(0), args['n_z'])
                    fake_data = decoder(z)
                    outputs = discriminator(fake_data.detach())
                    d_loss_fake = criterion_d(outputs, fake_labels)

                    d_loss = d_loss_real + d_loss_fake
                    d_loss.backward()
                    optimizer_d.step()

                    # Treinar o gerador
                    optimizer_g.zero_grad()
                    outputs = discriminator(fake_data)
                    g_loss = criterion_g(fake_data, batch) + criterion_d(outputs, real_labels)
                    g_loss.backward()
                    optimizer_g.step()

                if args['save']:
                    torch.save(encoder.state_dict(), os.path.join(MODELO_DIR_DST, f'encoder_epoch{epoch}.pt'))
                    torch.save(decoder.state_dict(), os.path.join(MODELO_DIR_DST, f'decoder_epoch{epoch}.pt'))

        encoder.load_state_dict(torch.load(os.path.join(MODELO_DIR_DST, 'encoder_epoch99.pt')))
        decoder.load_state_dict(torch.load(os.path.join(MODELO_DIR_DST, 'decoder_epoch99.pt')))

        y = y.astype(np.int64)
        n_to_sample_dict, major_class_count = calculate_n_to_sample(y)

        X_synthetic_list = []
        y_synthetic_list = []
        for cl, n_samples in n_to_sample_dict.items():
            if n_samples > 0:
                X_synthetic, y_synthetic = G_SM1(X, y, n_samples, cl, encoder, decoder)
                X_synthetic_list.append(X_synthetic)
                y_synthetic_list.append(y_synthetic)

        if X_synthetic_list:
            X_synthetic_combined = np.concatenate(X_synthetic_list, axis=0)
            y_synthetic_combined = np.concatenate(y_synthetic_list, axis=0)
            X_combined = np.vstack((X, X_synthetic_combined))
            y_combined = np.hstack((y, y_synthetic_combined))
        else:
            X_combined = X
            y_combined = y

        classifiers = {
            'Decision Tree': DecisionTreeClassifier(),
            'Random Forest': RandomForestClassifier(),
            'Neural Network': MLPClassifier(),
            'KNN': KNeighborsClassifier(),
            'XGBoost': XGBClassifier()
        }


        optimized_classifiers = {}
        for classifier_name, classifier in classifiers.items():
            search = BayesSearchCV(classifier, param_spaces[classifier_name], n_iter=10, cv=3, n_jobs=-1, random_state=42)
            search.fit(X_train, y_train)
            optimized_classifiers[classifier_name] = search.best_estimator_

        for classifier_name, classifier in optimized_classifiers.items():
            classifier.fit(X_combined, y_combined)
            joblib.dump(classifier, os.path.join(MODELO_DIR, f'{csv_file}_{classifier_name}_fold{fold}_model.pkl'))

            y_pred_train = classifier.predict(X_combined)
            precision_train = precision_score(y_combined, y_pred_train, average=None)
            recall_train = recall_score(y_combined, y_pred_train, average=None)
            f1_train = f1_score(y_combined, y_pred_train, average=None)

            for class_label, precision, recall, f1 in zip(np.unique(y_combined), precision_train, recall_train, f1_train):
                reports_df_train = pd.concat([reports_df_train, pd.DataFrame({
                    'Classifier': classifier_name,
                    'Class': class_label,
                    'Fold': fold,
                    'Precision': precision,
                    'Recall': recall,
                    'F1-Score': f1
                }, index=[0])])

            y_pred_val = classifier.predict(X_val)
            precision_val = precision_score(y_val, y_pred_val, average=None)
            recall_val = recall_score(y_val, y_pred_val, average=None)
            f1_val = f1_score(y_val, y_pred_val, average=None)

            precision_val_list[classifier_name].append(precision_val)
            recall_val_list[classifier_name].append(recall_val)
            f1_val_list[classifier_name].append(f1_val)

    for classifier_name in classifiers:
        precision_val_avg = np.mean(precision_val_list[classifier_name], axis=0)
        recall_val_avg = np.mean(recall_val_list[classifier_name], axis=0)
        f1_val_avg = np.mean(f1_val_list[classifier_name], axis=0)

        for class_label, precision, recall, f1 in zip(np.unique(y), precision_val_avg, recall_val_avg, f1_val_avg):
            reports_df_val = pd.concat([reports_df_val, pd.DataFrame({
                'Classifier': classifier_name,
                'Class': class_label,
                'Precision': precision,
                'Recall': recall,
                'F1-Score': f1
            }, index=[0])])

    reports_df_train.to_csv(os.path.join(OUTPUT_DIR_TRAIN, f'{csv_file}'), index=False)
    reports_df_val.to_csv(os.path.join(OUTPUT_DIR_TEST, f'{csv_file}'), index=False)
